In [ ]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.configs import BASE_URL, URLS_FRAGMENTS, COLUMN_NAME
from utils.helper_func import build_complete_urls, fetch_data, combined_df, parse_admin_region, parse_sale_period, find_first_sale_time, sales_start_time

In [ ]:
urls = build_complete_urls(BASE_URL, URLS_FRAGMENTS)
print(urls)

In [ ]:
df = combined_df(urls, "20250709")

In [ ]:
df = df.rename(columns=COLUMN_NAME, inplace=False)
df

In [ ]:
proc_df = df.copy()
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
proc_df["自售期間"], proc_df["代銷期間"] = zip(*proc_df["銷售期間"].apply(parse_sale_period))

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始時間"] = proc_df["自售期間"].apply(find_first_sale_time)
proc_df["代銷起始時間"] = proc_df["代銷期間"].apply(find_first_sale_time)

# 建立「銷售起始時間」欄位
proc_df["銷售起始時間"] = proc_df.apply(sales_start_time, axis=1)

In [ ]:
final_df = proc_df[["縣市", "行政區", "起造人", "社區名稱", "戶數", "銷售起始時間", "銷售期間",  
"編號", "自售起始時間", "代銷起始時間", "備查完成日期", "建照核發日", "坐落街道", "主要用途", "使用分區", "建照執照", "經度", "緯度"]]
final_df

In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\sale_data\processed"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
csv_fn = "預售屋備查.csv"
out_path = os.path.join(output_dir,  csv_fn)
final_df.to_csv(out_path, index=False, encoding='utf-8-sig')

pkl_fn   = "預售屋備查.pkl"
pkl_path = os.path.join(output_dir, pkl_fn)
final_df.to_pickle(pkl_path)